In [19]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [20]:
# Import the data
batchsize = 8
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)


In [21]:
train_data = train_datagen.flow_from_directory(r'C:\Users\INFINIX\Driver Drowsiness Detection System\Prepared_Data\train', target_size=(80,80),
                                               batch_size=batchsize, class_mode='categorical', subset='training')

validation_data = train_datagen.flow_from_directory(r'C:\Users\INFINIX\Driver Drowsiness Detection System\Prepared_Data\train', target_size=(80,80),
                                               batch_size=batchsize, class_mode='categorical', subset='validation')

Found 64105 images belonging to 2 classes.
Found 16025 images belonging to 2 classes.


In [22]:
test_datagen= ImageDataGenerator(rescale= 1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\INFINIX\Driver Drowsiness Detection System\Prepared_Data\test', target_size=(80,80),
                                               batch_size=batchsize, class_mode='categorical')

Found 4768 images belonging to 2 classes.


In [23]:
# Model training
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs=hmodel)

for layer in bmodel.layers:
    layer.trainable = False

In [24]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 80, 80, 3)]          0         []                            
                                                                                                  
 conv2d_94 (Conv2D)          (None, 39, 39, 32)           864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_94 (Ba  (None, 39, 39, 32)           96        ['conv2d_94[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_94 (Activation)  (None, 39, 39, 32)           0         ['batch_normalization_94

In [25]:
checkpoint = ModelCheckpoint(
    r'C:\Users\INFINIX\Driver Drowsiness Detection System\models\model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=3
)

earlystop = EarlyStopping(
    monitor='val_loss',
    patience=7,
    verbose=3,
    restore_best_weights=True
)

learning_rate = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    verbose=3,
)

callbacks = [checkpoint, earlystop, learning_rate]

In [26]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(
    train_data,
    steps_per_epoch=train_data.samples//batchsize,
    validation_data=validation_data,
    validation_steps=validation_data.samples//batchsize,
    callbacks=callbacks,
    epochs=5
)


C:\Users\INFINIX\AppData\Local\Temp\ipykernel_3516\2642282653.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/5
8013/8013 [==============================] - ETA: 0s - loss: 0.2079 - accuracy: 0.9180
Epoch 1: val_loss improved from inf to 0.19870, saving model to C:\Users\INFINIX\Driver Drowsiness Detection System\models\model.h5


C:\Users\INFINIX\.conda\envs\yogesh\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8013/8013 [==============================] - 1391s 172ms/step - loss: 0.2079 - accuracy: 0.9180 - val_loss: 0.1987 - val_accuracy: 0.9164 - lr: 0.0010
Epoch 2/5
 298/8013 [>.............................] - ETA: 17:50 - loss: 0.1826 - accuracy: 0.9237

KeyboardInterrupt: 

In [27]:
# Extract features for Random Forest
train_features = model.predict(train_data)
train_labels = train_data.classes

test_features = model.predict(test_data)
test_labels = test_data.classes

596/596 [==============================] - 69s 116ms/step


In [28]:
# Train Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(train_features, train_labels)
rf_predictions = rf_model.predict(test_features)
rf_accuracy = accuracy_score(test_labels, rf_predictions)

In [29]:
print("Random Forest Accuracy:", rf_accuracy)

Random Forest Accuracy: 0.5002097315436241
